## **LangGraph의 대화 히스토리 관리 방법** 

### MemorySaver와 Thread ID 
랭그래프에서 에이전트가 대화 맥락을 기억하게 만드는 과정은 크게 **[장치 장착: MemorySaver]** 과 **[사물함 지정: Thread ID]** 두 단계로 나뉩니다.
| 구분 | 컴파일 (`compile`) | 실행 (`invoke`) |
| --- | --- | --- |
| **비유** | 아파트에 **관리 사무소** 세우기 | **특정 호수(101호)** 에 짐 넣기 |
| **준비 코드** | `checkpointer=MemorySaver()` | `config={'configurable': {'thread_id': '...'}}`|
| **결과** | 기억할 준비가 된 그래프 생성 | `thread_id`별로 독립된 대화 기록 관리 |
| **이름 변경** | (거의 변경 안 함) | 이름을 바꾸면 **새로운 대화방**이 시작됨 |

즉, 똑같은 `graph`라도 `invoke` 할 때 `thread_id` 이름표를 다르게 붙여주면, 에이전트는 각각을 완전히 다른 사람(혹은 다른 주제)과의 대화로 인식하고 따로따로 기억하는 것이 핵심 

In [ ]:
''' 
1단계: 그래프 컴파일 (장치 장착)
에이전트에게 "기억할 수 있는 능력"을 부여하는 단계입니다.

- 역할: 에이전트에게 "기억 저장용 하드디스크"를 달아주는 것과 같습니다.
- 특징: 이 단계에서는 누구의 기억을 저장할지는 아직 모릅니다. 그냥 "저장할 준비"만 된 상태입니다.
'''

from langgraph.graph import MessagesState, StateGraph
from langgraph.checkpoint.memory import MemorySaver

graph_builder = StateGraph[MessagesState, None, MessagesState, MessagesState](MessagesState)

# 1. 기억 저장소(창고) 객체를 만듭니다.
checkpointer = MemorySaver()

# 2. 그래프를 컴파일할 때 이 저장소를 연결합니다.
graph = graph_builder.compile(
    checkpointer = checkpointer
)

In [ ]:
'''
2단계: 그래프 실행 (사물함 지정)
실제로 대화를 나눌 때, config를 통해 대화를 저장해야 하는 "사물함 번호"를 알려주는 단계입니다.

- 역할: 'MemorySaver'라는 큰 창고 안에서 특정 칸('thread_id')을 지정하는 것입니다.
- 작동 원리
    1. 로드(Load): 'thread_id' 칸을 열어 이전 대화(MessagesState)가 있는지 확인하고 가져옵니다.
    2. 실행(Run): 이전 대화 + 새로운 질문을 합쳐서 에이전트가 사고합니다.
    3. 저장(Save): 대화가 끝나면 새로 업데이트된 전체 메시지 리스트를 다시 그 칸에 저장합니다.
'''

from langchain.messages import HumanMessage

# 3. 특정 대화방을 구분할 번호표(thread_id)를 정합니다.
config = {
    'configurable': {
        'thread_id': 'paper_summary_01'  # 대화방 별 고유 이름
    }
}

# 4. 실행할 때 이 config를 함께 전달합니다.
graph.invoke(
    {'messages': [HumanMessage(content="질문 내용..")]}, 
    config=config #적용할 config 정보 입력 
)


## **MemorySaver의 기억 유효기간**
| 구분 | 1. MemorySaver 미사용 | 2. MemorySaver 사용 | 3. DB (Sqlite 등) 사용 |
| :--- | :--- | :--- | :--- |
| **기억 유효기간** | 함수 종료 시 휘발 | **프로그램 실행 중** (프로세스 종료 시 휘발) | **영구적** (데이터 삭제 전까지 유지) |
| **주요 용도** | 일회성 질문 응답 (기억 필요 없음) | 개발 및 로직 테스트용 | 실제 서비스 배포 및 운영용 |
| **특이사항** | 이전 대화 소환 불가능 | 서버 껐다 켜면 기억 상실 | 컴퓨터 꺼도 `thread_id`로 소환 가능 |


## **LangGraph에서 MemorySaver의 작동 방식**

### 1. 랭그래프의 기본 로직 (상태 업데이트)
- MemorySaver가 없어도 랭그래프가 노드를 실행할 때마다 기본적으로 수행하는 일입니다.
- 수행 주체: LangGraph Engine
- 작동 방식: 노드(Agent, Tools 등)가 결과를 반환하면, 그 값을 State 바구니에 합칩니다.
    - 예: Agent가 AIMessage를 반환 → MessagesState의 리스트에 append됨.
- 특징: 이 과정은 메모리(RAM) 상의 일시적인 작업입니다. 프로그램이 종료되거나 graph.invoke가 끝나면 이 바구니는 사라집니다.

### 2. MemorySaver의 추가 작업 (상태 기록)
- 컴파일 시 checkpointer를 등록했을 때만 활성화되는 "기록 서비스"입니다.
- 수행 주체: MemorySaver (Checkpointer)
- 작동 방식: 랭그래프가 위 1번 과정(업데이트)을 마칠 때마다, MemorySaver를 호출합니다.
    - "지금 방금 업데이트된 이 바구니(State) 전체를 스냅샷 찍어서 thread_id 칸에 저장해!"
- 특징: 단순한 업데이트를 넘어, **특정 시점의 상태를 '박제'** 하는 과정입니다.

### 3. 둘의 협력 프로세스
- 노드 실행: Agent가 답변을 생성함.
- → 기본 로직 (Update): 랭그래프가 답변을 messages 리스트에 **append** 함. (바구니 갱신)
- → 저장 로직 (Save): MemorySaver가 갱신된 바구니를 통째로 가져가서 thread_id 라벨을 붙여 따로 저장함. (스냅샷 기록)

| 구분 | 1. Update (기본 로직) | 2. Save (MemorySaver) |
| :--- | :--- | :--- |
| **공간** | 현재 실행 중인 **작업 메모리(RAM)** | MemorySaver 내부의 **데이터 저장소** |
| **형태** | 실시간으로 변하는 `MessagesState` 객체 | `thread_id`별로 관리되는 **상태 스냅샷(복사본)** |
| **관계** | **실행의 주체** (지금 당장 필요한 데이터) | **기록의 주체** (나중에 다시 꺼내올 데이터) |
| **동작** | 리스트에 `append` 함 | `thread_id` 칸에 통째로 `write` 함 |